In [1]:
#!/usr/bin/env python3
"""
THIRD SET COLAB 5: Continued Pretraining with TinyStories
Uses Unsloth for efficient training
Dataset: TinyStories (story generation)
Model: SmolLM2-135M
"""

print("="*80)
print("🚀 THIRD SET COLAB 5: CONTINUED PRETRAINING - TINYSTORIES")
print("="*80)

# INSTALL
print("\n📦 Installing...")
import subprocess
subprocess.run("pip install -q unsloth", shell=True)
subprocess.run("pip uninstall -y unsloth-colab xformers-local flash-attn", shell=True)
print("✅ Done!")

# IMPORTS
print("\n📚 Importing...")
from unsloth import FastLanguageModel
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
import torch

print("✅ Imported!")
print(f"GPU: {torch.cuda.is_available()}")

# LOAD MODEL
print("\n📥 Loading model...")
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/SmolLM2-135M-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
)
print("✅ Loaded!")

# ADD LORA
print("\n🔧 Adding LoRA...")
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
)
print("✅ LoRA added!")

# LOAD TINYSTORIES
print("\n📚 Loading TinyStories dataset...")
dataset = load_dataset("roneneldan/TinyStories", split="train[:500]")
print(f"✅ Loaded {len(dataset)} stories!")

# FORMAT
print("\n🔧 Formatting for continued pretraining...")

def format_stories(examples):
    texts = []
    for i in range(len(examples['text'])):
        story = examples['text'][i]
        # Use first 400 characters of each story
        if story.strip() and len(story) > 100:
            texts.append(story[:400] + tokenizer.eos_token)
    return {"text": texts}

formatted = format_stories(dataset)
# Filter
valid_texts = [t for t in formatted["text"] if len(t) > 100]

from datasets import Dataset as HFDataset
train_dataset = HFDataset.from_dict({"text": valid_texts[:400]})

print(f"✅ Formatted {len(train_dataset)} stories!")
print(f"\nExample story:\n{train_dataset[0]['text'][:200]}...")

# TRAINING
print("\n🚀 Continued pretraining on stories...")
print("Teaching model narrative and story structure...")

training_args = TrainingArguments(
    output_dir="./pretrain_stories",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    warmup_steps=5,
    logging_steps=10,
    save_strategy="no",
    fp16=True,
    report_to="none",
    max_steps=35,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=training_args,
)

trainer.train()
print("\n✅ Continued pretraining complete!")

# INFERENCE
print("\n🧪 Testing story generation...")
FastLanguageModel.for_inference(model)

story_starters = [
    "Once upon a time, there was a little girl who loved ",
    "In a magical forest, a brave knight ",
    "The curious cat decided to ",
]

for starter in story_starters:
    print(f"\n{'='*60}")
    print(f"Story starter: {starter}")
    print("Continuation:", end=" ")

    inputs = tokenizer(starter, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=60,
        temperature=0.8,
        use_cache=True,
        pad_token_id=tokenizer.eos_token_id
    )

    story = tokenizer.decode(outputs[0], skip_special_tokens=True)
    continuation = story[len(starter):].strip()
    print(continuation)

# SAVE
print("\n💾 Saving...")
model.save_pretrained("./stories_pretrained_model")
tokenizer.save_pretrained("./stories_pretrained_model")

print("\n" + "="*80)
print("🎉 THIRD SET COLAB 5 COMPLETE!")
print("="*80)
print("Summary:")
print("  ✓ Model: SmolLM2-135M")
print("  ✓ Dataset: TinyStories (400 stories)")
print("  ✓ Method: Continued pretraining")
print("  ✓ Learned: Story structure & narrative")
print("="*80)
print("\n🎊 ALL 5 THIRD SET COLABS COMPLETE! 🎊")
print("="*80)

🚀 THIRD SET COLAB 5: CONTINUED PRETRAINING - TINYSTORIES

📦 Installing...
✅ Done!

📚 Importing...
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
✅ Imported!
GPU: True

📥 Loading model...
==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/112M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

✅ Loaded 500 stories!

🔧 Formatting for continued pretraining...
✅ Formatted 400 stories!

Example story:
One day, a little girl named Lily found a needle in her room. She knew it was difficult to play with it because it was sharp. Lily wanted to share the needle with her mom, so she could sew a button on...

🚀 Continued pretraining on stories...
Teaching model narrative and story structure...


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/400 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 400 | Num Epochs = 1 | Total steps = 35
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 1,843,200 of 136,358,784 (1.35% trained)


Step,Training Loss
10,2.562800
20,2.467700
30,2.350100



✅ Continued pretraining complete!

🧪 Testing story generation...

Story starter: Once upon a time, there was a little girl who loved 
Continuation: to play with her toys. One day, she found a box of 
rubber gloves. She opened the box and saw a little boy 
who was playing with a ball. She was so happy that she 
took the ball and ran to the little boy. The little

Story starter: In a magical forest, a brave knight 
Continuation: took a sword and a shield.  The knight was a 
wonderful knight, and he was a very brave knight.  He 
took a sword and a shield and he went to the 
wonderful forest.  He saw a tree and he saw

Story starter: The curious cat decided to 
Continuation: take a walk. He was very careful, for he was afraid of 
the cat-squirrel. He was very careful, for he was afraid 
of the cat-squirrel. He was very careful, for he was afraid 
of the cat-squir

💾 Saving...

🎉 THIRD SET COLAB 5 COMPLETE!
Summary:
  ✓ Model: SmolLM2-135M
  ✓ Dataset: TinyStories (400 stories)
  ✓ Method: